<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### Removing Tags 

In [1]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python
!pip install compound-word-splitter
!pip install pyenchant
!sudo apt-get install enchant
!sudo apt-get install hunspell-it

     |████████████████████████████████| 112kB 5.7MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 184kB 7.0MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=c942b063b345a5e15d3d52ac36e3c144bfec8daf61e96f3a19471f9c3ca75938
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=7b543c8aa2cf35a2e9fec34bfb64de0cb445be64c0eba3d2ed77a8e406229107
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy
     |████████████████████████████████| 542kB 5.8MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp37-none-any.whl size=541554 sha256=3a34d4877d7db9a81659ffaa4e902f878dd6804732bfd300d1f090a528780453
  Stored in directo

In [2]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import splitter
import enchant
from itertools import groupby
import string

import sys
import os


In [4]:
# directory name:
input_dir = '/content/drive/My Drive/HLT/dataset_test_evalita/'
output_dir = '/content/drive/My Drive/HLT/dataset_test_evalita_preprocessed/'
input_dir_preprocessing = '/content/drive/My Drive/HLT/preprocessing/'

# Spec
pd.set_option("display.max_colwidth", None)

In [3]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
tsv_tweets_file = open(input_dir+"haspeede2_reference_taskAB-tweets.tsv")
#tsv_news_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-news.tsv")
#tsv_file = open(input_dir+"haspeede2_dev_taskAB.tsv")

#dataset_tweets_raw = pd.read_csv(tsv_tweets_file,sep='\t')
#dataset_tweets_raw.rename(columns={"text ": "text"}, inplace=True)
#dataset_news_raw = pd.read_csv(tsv_news_file,sep='\t')
#dataset_news_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw = pd.read_csv(tsv_tweets_file,sep='\t', header=None)
#dataset_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw.rename(columns={0: "id"}, inplace=True)
dataset_raw.rename(columns={1: "text"}, inplace=True)
dataset_raw.rename(columns={2: "hs"}, inplace=True)
dataset_raw.rename(columns={3: "stereotype"}, inplace=True)

#Bad Words
f2 = open(input_dir_preprocessing+"bad_words.csv", 'r', encoding='utf8')

bad_words_set = [] #list of lowercase words

for x in f2:
    y = x.rstrip()
    y = y.lower()
    if y != '':
        bad_words_set.append(y)

In [17]:
dataset_raw

,id,text,hs,stereotype
0,11834,"@user A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente in favore dell'immigrazione incontrollata...Meglio così, spariranno più velocemente!",1,0
1,12113,"@user e' un perfetto musulmano!!! chi dice il contrario mente, applica alla lettera la sura ""la donna"" del Corano. Il pericolo non e' lui in quanto tale, che lo dice, ma gli infami bugiardi! dell' islam moderato che NON ESISTE,",1,1
2,11770,"Mai Tg e i giornaloni hanno parlato di questa ennesima porcata fatta da immigrati ?? ⤵️⤵️⤵️⤵️ Sanremo, Segregata e stuprata per mesi. La giovane sequestrata da sei immigrati. Legata al letto e stuprata a turno per due mesi. Poi ceduta ad altri ""clienti"" URL",1,1
3,11937,@user Ipocriti farabutti. Fanno morire i terremotati per I Bastardi clandestini immigrati schifosi.,1,1
4,11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN CALO FISIOLOGICO QUANTI RINPATRII HA FATTO ? CI SONO MIGLIAIA DI IMMIGRATI IRREGOLARI CHE DELINQUONO TUTTI I GIORNI SUL TERRITORIO ITALIANI NON POSSONO ESSERE PUNITI DALLA GIUSTIZIA IN QUANTO PROFUGHI QUINDI ?,1,1
...,...,...,...,...
1258,10091,"Il Razzismo gli viene inculcato sin da bambini dal corano e dall'islam Cani, Maiali. Cristiani ed Ebrei sono impuri, infedeli e da sterminare. Il nemico e gran Pezzo di Merda è colui che li vuole qui a tutti i costi.",1,1
1259,10046,"PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? INFORMAZIONE CORROTTA,LA SINISTRA CONTINUA A COPRIRE I DELINQUENTI STRANIERI !! #viterbo #casapound #migranti URL",1,1
1260,10377,"FERMIAMO L'IMPOSTORE. Forse saranno necesarie altre CROCIATE, visto che il fiume di fango non si ferma.Anzi Vaticano, SX e POTERI MASSONICI ci stanno massacrando. Quando mai in Italia qualcuno veniva SGOZZATO, PRESO A SPRANGATE, FATTO A PEZZI? Fermiamo l'invasione dell'ISLAM!!!",1,1
1261,10199,@user @user @user @user @user @user @user @user @user @user Matteo non si tocca...fuori i delinquenti stranieri da casa nostra !,1,1


# Preprocessing phase
Following the "Preprocessing" section of https://books.openedition.org/aaccademia/4832?lang=it#tocfrom1n4 we will perform:
*   List item
*   List item



### Extraction of the first feature: length of the comment

In [18]:
def comment_length(text):
    return len(text)

In [19]:
dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)

Extraction of the second feature: percentage of words written in CAPS-LOCK inside the comment.

In [20]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [21]:
dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)

 ### Removing Tags 

In [22]:
def clean_tag(text):
    return re.sub(
        r'@user', ' ', text)

In [23]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_tag)

### Replace the characters ‘&’, ‘@’ respectively in the letters ‘e’, ‘a’

In [24]:
def replace_e_a(text):
    text = re.sub(r'&', 'e', text)
    return re.sub(r'@', 'a', text)

In [25]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_e_a)

### Conversion of disguised bad words

In [26]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' ca[.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

In [27]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)

### Hashtag splitting and saving

In [28]:
def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

In [29]:
dataset_raw['hashtags'] = dataset_raw['text'].apply(find_hashtags)

In [30]:
def split_hashtags(text):
    
    text = ' ' + text + ' '
    result = re.findall(r'#\S+', text)
    
    for word in result:
        new_word = " ".join(splitter.split(word[1:].lower(), 'it_IT'))
        if len(new_word)==0:
            new_word =  word[1:]
                  
        text = text.replace(word, new_word)
        
    return text

In [31]:
dataset_raw['text'] = dataset_raw['text'].apply(split_hashtags)

 ### Removing URL 

In [32]:
def clean_URL(text):
    return re.sub(r'URL', ' ', text)

In [33]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_URL)

### Extraction of the fourth feature: number of ‘?’ or ‘!’ inside the comment.


In [34]:
def esclamations_and_questions(text):
    return text.count('!') + text.count('?')

In [35]:
dataset_raw['#?!'] = dataset_raw['text'].apply(esclamations_and_questions)

### Extraction of the fifth feature: number of ‘.’ or ‘,’ inside the comment.


In [36]:
def esclamations_and_questions(text):
    return text.count(',') + text.count('.')

In [37]:
dataset_raw['#.,'] = dataset_raw['text'].apply(esclamations_and_questions)

### Punctuation removal

In [38]:
def strip_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [39]:
dataset_raw['text'] = dataset_raw['text'].apply(strip_punctuation)

### Removal of nearby equal vowels, removal of nearby equal consonants if they are more than 2

In [40]:
correct_words_vowels = ['sciiti,',
 'welcomerefugees',
 'livetweet',
 'desiree',
 'canaan',
 'tweet.',
 'weekend',
 'romafeyenoord',
 'ebree,',
 'greencard',
 'creerà',
 'cooperative.',
 'moschee,',
 'cooperanti',
 'streetart',
 'khalidmasood',
 'tweet',
 'woolfe',
 'cooperazione',
 'coop',
 'seehofer',
 'speech',
 'coffee',
 'scooter',
 'street',
 'veemenza',
 'moschee.',
 'maalox.',
 'book',
 'tweet',
 'facebook:',
 'sociial,',
 'coop,',
 'canaaniti.',
 'europee,',
 'cooperative',
 'google',
 'creeranno',
 'mediterranee',
 'cooperazione',
 'cooperativa',
 '“boom”',
 'refugees',
 'moonlight',
 'imaam',
 'shooting',
 'sciiti',
 'sunniti',
 'book',
 'atee.',
 'looking',
 'week',
 'ayaan',
 'temporanee.',
 'idee.',
 'sibiliini',
 'food',
 'refugees',
 'retweeted',
 'boom',
 'keep',
 'vodoo',
 'hooligans',
 'ebree',
 'refugees',
 'speed',
 'bloomberg',
 'riina',
 'hatespeech',
 'google',
 'masood',
 'linee.',
 'boom']

In [41]:
def delete_duplicate_vowels_and_redundant_consonant (text):
    parole = text.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]    
        vocali = ['a','e','i','o','u','y']
        
        for idx,val in enumerate(a):
            if idx == 0:
                stringa += a[idx][0] 
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola.lower() not in correct_words_vowels):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
        
    return(stringa)  

In [42]:
dataset_raw['text'] = dataset_raw['text'].apply(delete_duplicate_vowels_and_redundant_consonant)

### Translation of emoticons

In [43]:
def translate_emoticon(text):
    text_result = emoji.demojize(text, language='it')
    text_result=re.sub(r':', ' ', text_result)
    return text_result

In [44]:
dataset_raw['text'] = dataset_raw['text'].apply(translate_emoticon)

In [45]:
emoticons_text = {
    '<kiss>': 'bacio',
    '<happy>': 'felice',
    '<laugh>': 'risata',
    '<sad>': 'triste',
    '<surprise>': 'sorpreso',
    '<wink>': 'occhiolino',
    '<tong>': 'faccia con lingua',
    '<annoyed>': 'annoiato',
    '<seallips>': 'labbra sigillate',
    '<angel>': 'angelo',
    '<devil>': 'diavolo',
    '<highfive>' : 'batti il cinque',
    '<heart>': 'cuore',
    '<user>' : 'persona',
}

In [46]:
def clean_emoticon_text(text):
    text_words = text.split()
    new_words  = [emoticons_text.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

In [47]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_emoticon_text)

### Replacement of the abbreviations with the respective words

In [48]:
abbr_word = {'cmq':'comunque', 'gov':'governatori', 'fb':'facebook', 'tw':'twitter', 'juve':'juventus', 'ing':'ingegnere', 
             'sx':'sinistra', 'qdo':'quando', 'rep':'repubblica', 'grz':'grazie', 'ita':'italia', 'mln':'milioni', 
             'mld':'miliardi', 'pke':'perche', 'anke':'anche', 'cm':'come', 'dlla':'della', 'dlle':'delle', 'qst':'questa',
             'ke':'che', 'nn':'non', 'sn':'sono', 'cn':'con', 'xk':'perche', 'xke':'perche', 'art':'articolo',
             'tv':'televisore', '€':'euro', 'xché':'perché', 'xké':'perché', 'pkè':'perché'}

In [49]:
def replace_abbreviation(text):
    text_words = text.split()
    new_text = ""
    for token in text_words:
      new_text  += abbr_word.get(token, token) +" "
    
    return new_text.strip()

In [50]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_abbreviation)

### Removal of the laughs

In [51]:
laughs = ['ah', 'eh', 'he' 'ih', 'hi'] #non elimina ahahahah, ma solo ah
vowels = ['a', 'e', 'i', 'o', 'u']

def clean_laughs(text):
    #s = "ahahahah ho fame io, eh eh" -> " ho fame io,"
    text_words = text.split()
    new_words  = [word for word in text_words if word not in laughs]
    
    new_text = ' '.join(new_words)
    
    for i in new_words:
        for k in vowels:
            if ('h' in i) and (len(i) >= 4):
                if (len(i) - 2) <= (i.count(k) + i.count('h')):
                    new_text = new_text.replace(i, '')
    
    return new_text

In [52]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_laughs)

### Tokenization

In [53]:
def tokenization(text):
    tknzr=SocialTokenizer(lowercase=False)
    return tknzr.tokenize(text)

In [54]:
dataset_raw['tokens'] = dataset_raw['text'].apply(tokenization)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


### Extraction of the sixth feature: number of bad words in the comment

In [55]:
def number_bad_words(tokens):
    n_bad_words = 0

    for word in tokens:
        if word.lower() in bad_words_set:
            n_bad_words = n_bad_words + 1
        
    return n_bad_words

In [56]:
dataset_raw['#bad_words'] = dataset_raw['tokens'].apply(number_bad_words)

### Extraction of the seventh feature: percentage of bad words.

In [57]:
def percentage_bad_words(tokens):
    n_words = 0
    n_bad_words = 0
    
    for word in tokens:
        if word != '<' and word != '>':
            n_words = n_words + 1
    
    for word in tokens:
        if word.lower() in bad_words_set:
            n_bad_words = n_bad_words + 1
        
    return ((n_bad_words*100)//n_words)

In [58]:
dataset_raw['%bad_words'] = dataset_raw['tokens'].apply(percentage_bad_words)

In [59]:
dataset_raw.head(30)

,id,text,hs,stereotype,text_length,#C-L words,hashtags,#?!,"#.,",tokens,#bad_words,%bad_words
0,11834,A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente in favore dellimmigrazione incontrollataMeglio così spariranno più velocemente,1,0,180,4,None,1,4,"[A, me, pare, una, scelta, politica, suicida, puntare, tutto, su, una, battaglia, sicuramente, perdente, in, favore, dellimmigrazione, incontrollataMeglio, così, spariranno, più, velocemente]",0,0
1,12113,e un perfetto musulmano chi dice il contrario mente applica alla lettera la sura la donna del Corano Il pericolo non e lui in quanto tale che lo dice ma gli infami bugiardi del islam moderato che NON ESISTE,1,1,227,5,None,4,5,"[e, un, perfetto, musulmano, chi, dice, il, contrario, mente, applica, alla, lettera, la, sura, la, donna, del, Corano, Il, pericolo, non, e, lui, in, quanto, tale, che, lo, dice, ma, gli, infami, bugiardi, del, islam, moderato, che, NON, ESISTE]",0,0
2,11770,Mai Tg e i giornaloni hanno parlato di questa ennesima porcata fatta da immigrati freccia_curva_in_basso freccia_curva_in_basso freccia_curva_in_basso freccia_curva_in_basso Sanremo Segregata e stuprata per mesi La giovane sequestrata da sei immigrati Legata al letto e stuprata a turno per due mesi Poi ceduta ad altri clienti,1,1,259,2,None,2,4,"[Mai, Tg, e, i, giornaloni, hanno, parlato, di, questa, ennesima, porcata, fatta, da, immigrati, freccia_curva_in_basso, freccia_curva_in_basso, freccia_curva_in_basso, freccia_curva_in_basso, Sanremo, Segregata, e, stuprata, per, mesi, La, giovane, sequestrata, da, sei, immigrati, Legata, al, letto, e, stuprata, a, turno, per, due, mesi, Poi, ceduta, ad, altri, clienti]",1,2
3,11937,Ipocriti farabutti Fanno morire i terremotati per I Bastardi clandestini immigrati schifosi,1,1,99,7,None,0,2,"[Ipocriti, farabutti, Fanno, morire, i, terremotati, per, I, Bastardi, clandestini, immigrati, schifosi]",0,0
4,11870,LIMMIGRAZIONE CE STATO UN CALO FISIOLOGICO QUANTI RINPATRI HA FATTO CI SONO MIGLIAIA DI IMMIGRATI IRREGOLARI CHE DELINQUONO TUTTI I GIORNI SUL TERRITORIO ITALIANI NON POSSONO ESSERE PUNITI DALLA GIUSTIZIA IN QUANTO PROFUGHI QUINDI,1,1,257,87,None,2,0,"[LIMMIGRAZIONE, CE, STATO, UN, CALO, FISIOLOGICO, QUANTI, RINPATRI, HA, FATTO, CI, SONO, MIGLIAIA, DI, IMMIGRATI, IRREGOLARI, CHE, DELINQUONO, TUTTI, I, GIORNI, SUL, TERRITORIO, ITALIANI, NON, POSSONO, ESSERE, PUNITI, DALLA, GIUSTIZIA, IN, QUANTO, PROFUGHI, QUINDI]",0,0
5,12122,SI CERTO E TUTTIQUELLI CHE GIRANO A VUOTO SENZA FARE UNCAZZO CE LI DOBBIAMO TENERE ESPATRIOOLE GALERE SONO PIENE 80 SONO STRANIERI LI DOBBIAMO MANTENERE ANCHE LÌ BASTA,1,1,210,90,None,7,12,"[SI, CERTO, E, TUTTIQUELLI, CHE, GIRANO, A, VUOTO, SENZA, FARE, UNCAZZO, CE, LI, DOBBIAMO, TENERE, ESPATRIOOLE, GALERE, SONO, PIENE, 80, SONO, STRANIERI, LI, DOBBIAMO, MANTENERE, ANCHE, LÌ, BASTA]",0,0
6,12095,Ha ragione É il corano che prescrive la lotta agli infedeli e loro sottomissione nessuna convivenza pacifica e men che mai paritetica,1,1,139,8,None,1,1,"[Ha, ragione, É, il, corano, che, prescrive, la, lotta, agli, infedeli, e, loro, sottomissione, nessuna, convivenza, pacifica, e, men, che, mai, paritetica]",0,0
7,11609,Per quanto mi riguarda laccogliere indiscriminatamente ci ha portato solo parassiti stupratori spacciatori e mafia nigeriana con tanto di traffico di organi umani e molta meno sicurezza Migranti Fico “L’accoglienza significa crescita e sicurezza”,1,1,258,2,None,2,1,"[Per, quanto, mi, riguarda, laccogliere, indiscriminatamente, ci, ha, portato, solo, parassiti, stupratori, spacciatori, e, mafia, nigeriana, con, tanto, di, traffico, di, organi, umani, e, molta, meno, sicurezza, Migranti, Fico, “, L, ’, accoglienza, significa, crescita, e, sicurezza, ”]",1,2
8,12141,IO DOMANDO A QUESTE MERDE che vogliono e accettano altre merde mussulmane se gli viene in mente cosa si preparerebbe per noi oppure sono solo merde che n pensano se no ai soldi non parliamo poi q questo auspicio è proclamato da 

In [60]:
dataset_raw.to_csv('test_dataset_tweet.csv', index=False)